# Analyse des causes d’attrition chez TechNova Partners  
## 1. Ingestion et contrôle qualité des données

**Projet :** HR Analytics – Identification des causes de démission  
**Rôle :** Consultant Data Scientist  
**Objectif du notebook :**  
Construire un DataFrame central propre et cohérent à partir de trois sources RH hétérogènes, en réalisant les premiers contrôles qualité indispensables avant toute analyse exploratoire ou modélisation.


## Contexte et problématique

TechNova Partners fait face à un taux de turnover plus élevé qu’habituellement.  
Le département RH souhaite identifier les causes potentielles de ces départs afin de mettre en place des leviers d’action ciblés.

Trois sources de données internes sont mises à disposition :

1. **SIRH** : informations contractuelles, sociodémographiques et professionnelles
2. **Évaluations annuelles** : performance et satisfaction
3. **Sondage interne** : bien-être, perception, et variable cible de démission

Avant toute analyse ou modélisation, il est indispensable de :
- comprendre la structure de chaque source,
- vérifier la qualité des données,
- identifier les clés de jointure,
- construire un jeu de données central cohérent et anonymisé.


## Démarche méthodologique

La démarche suivie dans ce notebook est la suivante :

1. Chargement des trois fichiers CSV bruts
2. Contrôle qualité initial (dimensions, types, valeurs manquantes, doublons)
3. Harmonisation des clés de jointure
4. Jointure des trois sources
5. Contrôles post-jointure
6. Anonymisation des identifiants employés
7. Sauvegarde d’un dataset central prêt pour l’analyse exploratoire

Les transformations récurrentes sont implémentées dans des fonctions Python réutilisables (dossier `src/`) afin de garantir :
- la reproductibilité,
- la lisibilité,
- et la maintenabilité du projet.


In [4]:
from technova_attrition.config import PATHS
from technova_attrition.data_io import (
    anonymize_employee_id,
    check_duplicates,
    join_sources,
    load_eval,
    load_sirh,
    load_sondage,
)

## Chargement des données brutes

Les fichiers CSV sont stockés dans le dossier `data/raw`.  
Ils correspondent aux extractions brutes fournies par le responsable SIRH.

À ce stade, aucune transformation métier n’est appliquée.


In [ ]:
sirh_path = PATHS.data_raw / "extrait_sirh.csv"
eval_path = PATHS.data_raw / "extrait_eval.csv"
sondage_path = PATHS.data_raw / "extrait_sondage.csv"

sirh_df = load_sirh(sirh_path)
eval_df = load_eval(eval_path)
sondage_df = load_sondage(sondage_path)

sirh_df.shape, eval_df.shape, sondage_df.shape

FileNotFoundError: CSV not found: G:\Mon Drive\OC\Projet_4\technova_attrition\data\raw\Extrait SIRH.csv

## Inspection initiale des jeux de données

Nous examinons pour chaque source :
- les dimensions (nombre de lignes et colonnes),
- les types de variables,
- un aperçu des premières lignes.

Cette étape permet de détecter rapidement :
- des incohérences de typage,
- des colonnes inutiles,
- des valeurs manifestement aberrantes.


In [ ]:
sirh_df.head()

In [ ]:
sirh_df.info()

## Recherche de doublons

Avant toute jointure, il est essentiel de vérifier l’unicité des identifiants métiers.

Nous recherchons ici :
- des doublons stricts (lignes identiques),
- des doublons sur les clés de jointure potentielles.

Ces vérifications permettent d’éviter des duplications artificielles lors des jointures.


In [ ]:
check_duplicates(sirh_df, subset=["id_employee"]).shape

In [ ]:
check_duplicates(eval_df, subset=["eval_number"]).shape

In [ ]:
check_duplicates(sondage_df, subset=["code_sondage"]).shape

## Harmonisation des clés de jointure

Les trois sources ne partagent pas une clé strictement identique :

- **SIRH** : `id_employee` (entier)
- **Évaluations** : `eval_number` (ex: "E_1", "E_2", ...)
- **Sondage** : `code_sondage` (entier)

Une transformation est nécessaire pour rendre ces clés compatibles :
- extraction de la partie numérique de `eval_number`
- création d’une clé numérique `eval_number_int`

Cette étape est indispensable pour garantir des jointures fiables.


In [ ]:
eval_df[["eval_number", "eval_number_int"]].head()

## Jointure des trois sources

Une jointure **inner** est utilisée afin de :
- conserver uniquement les employés présents dans les trois systèmes,
- garantir la cohérence du dataset final.

Cette décision est justifiée dans un contexte d’analyse explicative, où la qualité prime sur l’exhaustivité.


In [ ]:
df = join_sources(sirh_df, eval_df, sondage_df, how="inner")
df.shape

## Contrôles qualité post-jointure

Après la jointure, nous vérifions :
- l’absence de duplication inattendue,
- la cohérence des dimensions,
- la présence effective de la variable cible.

Ces contrôles permettent de s’assurer que la jointure n’a pas introduit d’erreurs structurelles.


In [ ]:
df.duplicated().sum()

In [ ]:
df.columns

## Anonymisation des identifiants employés

Les données RH sont **hautement sensibles**.  
Avant toute analyse ou modélisation, les identifiants employés doivent être anonymisés.

Méthode utilisée :
- **HMAC-SHA256** avec clé secrète
- anonymisation stable (un même employé garde le même identifiant anonymisé)
- non réversible sans la clé

Cette approche est conforme aux bonnes pratiques RGPD en contexte analytique.


In [ ]:
df["employee_id_anon"] = anonymize_employee_id(df["id_employee"])
df = df.drop(columns=["id_employee"])

df[["employee_id_anon"]].head()

## Sauvegarde du dataset central

Le DataFrame final est sauvegardé dans le dossier `data/processed`.

Ce fichier constitue :
- l’entrée unique pour l’analyse exploratoire,
- la base de travail pour le feature engineering,
- la référence utilisée pour la partie SQL et la modélisation.


In [ ]:
output_path = PATHS.data_processed / "employees_joined.parquet"
df.to_parquet(output_path, index=False)

output_path

## Conclusion de l’étape d’ingestion

À l’issue de ce notebook, nous disposons :
- d’un dataset central issu de trois sources RH,
- de données contrôlées et cohérentes,
- d’identifiants employés anonymisés,
- d’un fichier prêt pour l’analyse exploratoire et statistique.

La prochaine étape consistera à :
- analyser les distributions des variables,
- comparer les employés ayant quitté l’entreprise à ceux encore présents,
- identifier les premiers signaux explicatifs de l’attrition.
